# Download de Dados da Nuvem CAED

## 1. Imports e Configurações

In [ ]:
import pandas as pd
import requests
import openpyxl
import json
import os
from datetime import datetime
from sqlalchemy import create_engine, VARCHAR

In [ ]:
# ==================== CONFIGURAÇÕES ====================

# API
URL_BASE = 'https://parc.caeddigital.net/portal/classes'
HEADERS = {
    'X-Parse-Application-Id': 'portal',
    'X-Parse-Client-Key': 'js3.4.2',
    'X-Parse-Master-Key': '(daKjG_?x_xvba9',
    'X-Parse-Installation-Id': 'd35b5929-fcc4-45f4-8bed-44f219c2d533',
    'Content-Type': 'application/json',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

# Banco de Dados
SERVIDOR = '192.168.250.8,61433'
BANCO_DE_DADOS = 'SDV_TESTE_IMPORTACAO'
TRUSTED_CONNECTION = 'yes'

# Processamento
BATCH_SIZE = 10000
COLLECTIONS = ['E_1308_ESTADO']

# Configuração do Where
WHERE_CLAUSE = {"excluido": False}

## 2. Download dos Dados da API

In [ ]:
def obter_todos_dados_api(collection, batch_size=BATCH_SIZE):
    """
    Baixa todos os dados de uma collection da API.
    """
    url = f"{URL_BASE}/{collection}"
    
    all_results = []
    skip = 0
    total_retrieved = 0
    
    try:
        while True:
            payload = json.dumps({
                "where": WHERE_CLAUSE,
                "limit": batch_size,
                "skip": skip
            })
            
            response = requests.get(url, headers=HEADERS, data=payload)
            response.raise_for_status()
            
            data = response.json()
            results = data.get('results', [])
            
            if not results:
                break
                
            all_results.extend(results)
            total_retrieved += len(results)
            skip += batch_size
            
            print(f"Collection {collection}: Baixados {total_retrieved} registros...\n")
            
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a API para collection {collection}: {e}")
    
    return pd.json_normalize(all_results)

In [ ]:
# Baixar dados das collections
dataframes = {col: obter_todos_dados_api(col) for col in COLLECTIONS}

# # Criar variável de atalho para a collection principal
# enturmacao = dataframes['E_1308_ESTADO']

# # Mostrar informações
# display(enturmacao.head())
# print(f"\nTotal de registros: {len(enturmacao)}")

## 3. Preparação para Importação no SQL Server

In [ ]:
# Criar engine de conexão
string_conexao = f'mssql+pyodbc://{SERVIDOR}/{BANCO_DE_DADOS}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection={TRUSTED_CONNECTION}'
engine = create_engine(string_conexao, fast_executemany=True)

print("Conexão com banco de dados criada")

In [ ]:
# Calcular tamanhos máximos das colunas
tamanhos_maximos = {}
tipos_de_dados = {}

for nome_collection, df in dataframes.items():
    # Calcula tamanho máximo de cada coluna
    tamanhos = df.map(lambda x: len(str(x))).max()
    tamanhos_maximos[nome_collection] = tamanhos
    
    # Define tipo VARCHAR com o tamanho calculado
    tipos = {coluna: VARCHAR(tamanho) for coluna, tamanho in tamanhos.items()}
    tipos_de_dados[nome_collection] = tipos

print("Tipos de dados calculados")

## 4. Importação para SQL Server

In [ ]:
# Importar cada DataFrame para o SQL Server
for nome_collection, df in dataframes.items():
    print(f"\nImportando {nome_collection}...")
    
    # Converter colunas que contenham listas/dicionários em JSON (pyodbc/SQLAlchemy não aceita lista/dict direto)
    cols_to_convert = [col for col in df.columns if df[col].apply(lambda x: isinstance(x, (list, dict))).any()]
    for col in cols_to_convert:
        df[col] = df[col].apply(lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, (list, dict)) else x)
    
    # Recalcular tamanhos máximos e tipos para este DataFrame (pode ter mudado ao serializar JSON)
    # usamos .astype(str) para garantir que todos os valores sejam strings ao medir o tamanho
    tamanhos = df.astype(str).apply(lambda col: col.map(len).max())
    tipos_local = {col: VARCHAR(int(tamanho)) for col, tamanho in tamanhos.items()}
    tipos_de_dados[nome_collection] = tipos_local  # atualizar o mapa global se quiser
    
    df.to_sql(
        nome_collection,
        engine,
        index=False,
        if_exists='replace',  # Mude para 'replace' se quiser substituir a tabela
        dtype=tipos_local
    )
    
    print(f"✓ {nome_collection} importado com sucesso! ({len(df)} registros)")

print("\n" + "="*50)
print("IMPORTAÇÃO CONCLUÍDA COM SUCESSO!")
print("="*50)